In [1]:
from pathlib import Path
from IPython.display import HTML, display
css = Path("../../../css/custom.css").read_text(encoding="utf-8")
display(HTML(f"<style>{css}</style>"))

# Chapter 1 — Introduction to Machine Learning
# Lesson 10: Learning Paradigms and Task Taxonomy (Regression, Classification, Ranking, Forecasting)


## Chapter 1 — Introduction to Machine Learning
## Lesson 10: Learning Paradigms and Task Taxonomy (Regression, Classification, Ranking, Forecasting)


This notebook is part of a bilingual, notebook-first machine learning course.

### Why this lesson exists

Practitioners frequently start with algorithms ("Should we use XGBoost?") before clarifying the **task**.
That order is backwards. A clean task definition determines:

- what the target is ($y$),
- what data generation assumptions you can reasonably make (IID vs time-dependent),
- which metrics you will report,
- and what validation protocol is legitimate.

### Scope

We focus on the most common task families in classical ML:

1. Regression
2. Classification (binary + multi-class)
3. Ranking / Learning-to-Rank
4. Forecasting (supervised learning under temporal dependence)

We will also connect these to **learning paradigms** (supervised/unsupervised/semi-supervised/RL),
but the bulk of the hands-on work is supervised.

---

## Learning objectives

By the end of this lesson, you should be able to:

1. Map a business question to a *task type* (regression/classification/ranking/forecasting).
2. Choose appropriate **losses** and **metrics** for each task.
3. Recognize the differences between **IID supervised learning** and **time-dependent** forecasting evaluation.
4. Build minimal, correct baselines in scikit-learn and interpret results.
5. Explain common failure modes (metric mismatch, data leakage, invalid splits).

---

## A mental model: "Task = target type + evaluation protocol + decision"

A practical definition of a task is:

$$
\text{Task} = \big(\text{target type}, \; \text{valid evaluation}, \; \text{decision cost}\big)
$$

If you only define the target type and ignore evaluation, you'll likely ship a model that fails in production.


In [2]:
import io
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, f1_score, classification_report,
    confusion_matrix, roc_auc_score
)
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.ensemble import RandomForestRegressor

np.random.seed(0)

def read_csv_or_sample(path, sample_csv_text, **kwargs):
    """Read a CSV from repo path; fall back to embedded sample rows if missing."""
    p = Path(path)
    if p.exists():
        return pd.read_csv(p, **kwargs)
    return pd.read_csv(io.StringIO(sample_csv_text), **kwargs)

def synthesize_from_sample(df, n=500, noise=0.05, seed=42):
    """Bootstrap + jitter to make a larger dataset for demonstrative modeling."""
    rng = np.random.default_rng(seed)
    df_big = df.sample(n=n, replace=True, random_state=seed).reset_index(drop=True)
    for col in df_big.columns:
        if pd.api.types.is_numeric_dtype(df_big[col]):
            std = df_big[col].std(ddof=0)
            if np.isfinite(std) and std > 0:
                df_big[col] = df_big[col] + rng.normal(0, noise*std, size=len(df_big))
    return df_big

def synthesize_earthquake_timeseries(df_small, days=220, avg_events_per_day=4, seed=19):
    """Create a multi-day series when only sample rows are available."""
    rng = np.random.default_rng(seed)
    df_small = df_small.copy()
    df_small["timestamp"] = pd.to_datetime(df_small["date"].astype(str) + " " + df_small["time"].astype(str), errors="coerce")
    df_small = df_small.dropna(subset=["timestamp"])

    start = df_small["timestamp"].min().normalize()
    rows = []
    for d in range(days):
        day = start + pd.Timedelta(days=d)
        m = rng.poisson(avg_events_per_day) + 1
        sample = df_small.sample(n=m, replace=True, random_state=int(seed + d)).reset_index(drop=True)
        secs = rng.integers(0, 24*3600, size=m)
        sample["timestamp"] = day + pd.to_timedelta(secs, unit="s")

        for col in ["latitude", "longitude", "depth", "magnitude"]:
            if col in sample.columns:
                sample[col] = pd.to_numeric(sample[col], errors="coerce")
                std = np.nanstd(sample[col])
                if np.isfinite(std) and std > 0:
                    sample[col] = sample[col] + rng.normal(0, 0.10*std, size=m)

        rows.append(sample)

    out = pd.concat(rows, ignore_index=True)
    out["date"] = out["timestamp"].dt.date.astype(str)
    out["time"] = out["timestamp"].dt.time.astype(str)
    return out.drop(columns=["timestamp"])

def rmse(y_true, y_pred):
    """Compute RMSE without using deprecated sklearn squared=... parameter."""
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

print("Setup complete. Versions:")
import sklearn
print("  pandas:", pd.__version__)
print("  numpy:", np.__version__)
print("  sklearn:", sklearn.__version__)


Setup complete. Versions:
  pandas: 2.2.3
  numpy: 2.1.2
  sklearn: 1.5.2


## 1. Learning paradigms: where supervision comes from

A **learning paradigm** specifies how the model obtains information that defines "good behavior".

### 1.1 Supervised learning

You observe labeled pairs $(x_i, y_i)$ and learn a mapping $f: x \mapsto y$.

Typical supervised objective:

$$
\hat{f} = \arg\min_{f \in \mathcal{F}} \; \frac{1}{n} \sum_{i=1}^n \ell\big(y_i, f(x_i)\big) + \lambda \Omega(f)
$$

- $\ell$ is a *loss* (how wrong a prediction is)
- $\Omega$ is a regularizer (capacity control)
- $\lambda$ tunes the regularization strength

### 1.2 Unsupervised learning

You observe $x_i$ without explicit targets and learn structure: clusters, densities, embeddings.
Unsupervised learning is frequently used for feature learning, segmentation, and anomaly detection.

### 1.3 Semi-supervised and weak supervision

You have a small labeled set and a large unlabeled set.
You may use pseudo-labels or graph structure to propagate labels.
In practice, a "weak supervision" system can generate noisy labels from heuristic rules.

### 1.4 Reinforcement learning (RL) and bandits

You interact with an environment, choose actions, and receive rewards.
This is not the focus here, but note that some ranking/recommendation problems are treated as contextual bandits.

---

## 2. Task taxonomy: target type and evaluation protocol

A compact taxonomy for common supervised tasks:

| Task type | Target $y$ | Prediction output | Typical metric |
|---|---|---|---|
| Regression | $\mathbb{R}$ | value | MAE, RMSE, $R^2$ |
| Binary classification | $\{0,1\}$ | label/probability | F1, ROC AUC, PR AUC |
| Multi-class classification | $\{1,\dots,K\}$ | class probs | accuracy, macro-F1 |
| Ranking (L2R) | relevance / pairwise prefs | ordering | NDCG@k, MAP, Recall@k |
| Forecasting | $y_{t+h}$ | future value | MAE/RMSE by horizon |

Notice: **forecasting** is usually supervised but **not IID**.
That changes what "train/test split" means.

---

## 2.1 Cost, thresholds, and decision rules

Even for the same task type, costs can change the decision rule.

For binary classification, a common decision is:

$$
\hat{y} = \mathbf{1}\big[\; P(y=1\mid x) \ge \tau \;\big]
$$

The threshold $\tau$ should reflect false positive vs false negative costs,
not a default $0.5$.

For ranking, you often care about the *top* of the list because of limited bandwidth:

- You email the top 1% of leads
- You show the top 10 products
- You review the top 50 flagged transactions

So you should evaluate with top-$k$ metrics.

---

## 2.2 A small checklist for task framing

Before training anything, write down:

1. What is the target $y$ (including units)?
2. What action follows the prediction?
3. What is the cost of false positives/negatives or large errors?
4. What data splitting rule matches production?

We'll now make these ideas concrete with code examples.


## 3. Toy framing: regression vs classification vs ranking

This toy example uses a single feature $x$ and an underlying continuous outcome.

- In **regression**, you predict $y$ directly.
- In **classification**, you predict whether $y$ exceeds a threshold.
- In **ranking**, you might use the predicted score to order instances by risk.

Even though they use the same raw data, the learned parameters and the evaluation are different.


In [3]:
# A tiny illustration: the same data can be framed as regression or classification.

rng = np.random.default_rng(0)
n = 12
x = rng.normal(size=n)
y_cont = 2*x + rng.normal(scale=0.5, size=n)          # continuous target
y_bin = (y_cont > np.median(y_cont)).astype(int)      # binarized target

toy = pd.DataFrame({"x": x, "y_cont": y_cont, "y_bin": y_bin})
display(toy)

# Regression framing
reg = LinearRegression().fit(toy[["x"]], toy["y_cont"])
y_hat_reg = reg.predict(toy[["x"]])

# Classification framing
clf = LogisticRegression().fit(toy[["x"]], toy["y_bin"])
p_hat = clf.predict_proba(toy[["x"]])[:, 1]

print("Regression coefficients:", reg.coef_[0], "intercept:", reg.intercept_)
print("Classification coefficient:", clf.coef_[0,0], "intercept:", clf.intercept_[0])
print("\nFirst 5 predictions:")
for i in range(5):
    print(f"  x={x[i]: .3f}  y_cont={y_cont[i]: .3f}  y_hat_reg={y_hat_reg[i]: .3f}  p(y=1)={p_hat[i]: .3f}")


,x,y_cont,y_bin
0,0.125730,-0.911055,0
1,-0.132105,-0.373606,0
2,0.640423,0.657890,1
3,0.104900,-0.156333,1
4,-0.535669,-1.343468,0
5,0.361595,0.565040,1
6,1.304000,2.813815,1
7,0.947081,2.415419,1
8,-0.703735,-1.471738,0
9,-1.265421,-1.847611,0


Regression coefficients: 1.9512520123535053 intercept: -0.12409791654978095
Classification coefficient: 1.4488474737404589 intercept: -0.027935787302401996

First 5 predictions:
  x= 0.126  y_cont=-0.911  y_hat_reg= 0.121  p(y=1)= 0.538
  x=-0.132  y_cont=-0.374  y_hat_reg=-0.382  p(y=1)= 0.445
  x= 0.640  y_cont= 0.658  y_hat_reg= 1.126  p(y=1)= 0.711
  x= 0.105  y_cont=-0.156  y_hat_reg= 0.081  p(y=1)= 0.531
  x=-0.536  y_cont=-1.343  y_hat_reg=-1.169  p(y=1)= 0.309


## 4. Regression example: house price prediction

### 4.1 Problem statement

Given features about a house, predict its selling price:

- Inputs $x$: size, bedrooms, bathrooms, neighborhood, etc.
- Target $y$: price (currency)

This is a regression task because $y \in \mathbb{R}$.

### 4.2 Baseline modeling approach

We'll build:

1. A preprocessing pipeline:
   - numeric: median imputation + standardization
   - categorical: most-frequent imputation + one-hot encoding
2. A linear regression model

### 4.3 Metrics

We report MAE, RMSE, and $R^2$:

- MAE is interpretable in the target unit.
- RMSE penalizes large errors more.
- $R^2$ is useful for variance explained but can be misleading across datasets.

Let's implement the baseline.


In [4]:
# Regression dataset: house-prices
HOUSE_PATH = "../../../Datasets/Regression/house-prices.csv"
sample_house = "Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood\n1,114300,1790,2,2,2,No,East\n2,114200,2030,4,2,3,No,East\n3,114800,1740,3,2,1,No,East\n4,94700,1980,3,2,3,No,East\n5,119800,2130,3,3,3,No,East\n"

house_df = read_csv_or_sample(HOUSE_PATH, sample_house)
print("Raw rows (head):")
print(house_df.head().to_string(index=False))

df = synthesize_from_sample(house_df, n=500, noise=0.08, seed=13)

y = df["Price"].astype(float)
X = df.drop(columns=["Price"]).copy()

num_cols = ["SqFt", "Bedrooms", "Bathrooms", "Offers"]
cat_cols = ["Brick", "Neighborhood"]

pre = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
])

model = Pipeline([("pre", pre), ("linreg", LinearRegression())])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model.fit(X_train, y_train)
pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)
rmse_val = rmse(y_test, pred)
r2 = r2_score(y_test, pred)

print(f"MAE:  {mae:,.2f}")
print(f"RMSE: {rmse_val:,.2f}")
print(f"R^2:  {r2:.3f}")


Raw rows (head):
 Home  Price  SqFt  Bedrooms  Bathrooms  Offers Brick Neighborhood
    1 114300  1790         2          2       2    No         East
    2 114200  2030         4          2       3    No         East
    3 114800  1740         3          2       1    No         East
    4  94700  1980         3          2       3    No         East
    5 119800  2130         3          3       3    No         East
MAE:  7,283.09
RMSE: 9,529.77
R^2:  0.889


## 5. Binary classification example: diabetes prediction

### 5.1 Problem statement

Given patient measurements, predict whether the patient is diabetic.

- Inputs $x$: glucose, BMI, age, etc.
- Target $y \in \{0,1\}$: diabetic vs non-diabetic

### 5.2 Probabilistic prediction matters

Many downstream decisions depend on *risk* rather than a hard label.
Logistic regression returns an estimated probability $P(y=1\mid x)$.

### 5.3 Metrics

We'll compute:

- Accuracy
- F1
- ROC AUC
- Confusion matrix

In imbalanced datasets, accuracy alone can be misleading; F1 and AUC provide additional signal.


In [5]:
# Classification dataset: diabetes
DIAB_PATH = "../../../Datasets/Classification/diabetes.csv"
sample_diabetes = "Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,classification\n6,148,72,35,0,33.6,0.627,50,Diabetic\n1,85,66,29,0,26.6,0.351,31,Non-Diabetic\n8,183,64,0,0,23.3,0.672,32,Diabetic\n1,89,66,23,94,28.1,0.167,21,Non-Diabetic\n0,137,40,35,168,43.1,2.288,33,Diabetic\n"

diab_df = read_csv_or_sample(DIAB_PATH, sample_diabetes)
print("Raw rows (head):")
print(diab_df.head().to_string(index=False))

df = synthesize_from_sample(diab_df, n=600, noise=0.10, seed=7)

y = (df["classification"].astype(str) == "Diabetic").astype(int)
X = df.drop(columns=["classification"])

pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, random_state=0)),
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)

pipe.fit(X_train, y_train)
proba = pipe.predict_proba(X_test)[:, 1]
pred = (proba >= 0.5).astype(int)

acc = accuracy_score(y_test, pred)
f1 = f1_score(y_test, pred)
auc = roc_auc_score(y_test, proba)
cm = confusion_matrix(y_test, pred)

print(f"Accuracy: {acc:.3f}")
print(f"F1:       {f1:.3f}")
print(f"ROC AUC:  {auc:.3f}")
print("Confusion matrix [[TN, FP], [FN, TP]]:")
print(cm)


Raw rows (head):
 Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin  BMI  DiabetesPedigreeFunction  Age classification
           6      148             72             35        0 33.6                     0.627   50       Diabetic
           1       85             66             29        0 26.6                     0.351   31   Non-Diabetic
           8      183             64              0        0 23.3                     0.672   32       Diabetic
           1       89             66             23       94 28.1                     0.167   21   Non-Diabetic
           0      137             40             35      168 43.1                     2.288   33       Diabetic
Accuracy: 0.793
F1:       0.667
ROC AUC:  0.828
Confusion matrix [[TN, FP], [FN, TP]]:
[[88  8]
 [23 31]]


## 6. Multi-class classification: Iris

Multi-class classification predicts among $K>2$ classes.
Even when accuracy looks high, always inspect per-class performance with a classification report.

We use the iris dataset for a compact demonstration.


In [6]:
# Multi-class dataset: iris
IRIS_PATH = "../../../Datasets/Classification/iris.csv"
sample_iris = "sepal_length,sepal_width,petal_length,petal_width,classification\n5.1,3.5,1.4,0.2,Iris-setosa\n4.9,3.0,1.4,0.2,Iris-setosa\n5.0,3.6,1.4,0.2,Iris-setosa\n7.0,3.2,4.7,1.4,Iris-versicolor\n6.4,3.2,4.5,1.5,Iris-versicolor\n6.9,3.1,4.9,1.5,Iris-versicolor\n6.3,3.3,6.0,2.5,Iris-virginica\n5.8,2.7,5.1,1.9,Iris-virginica\n7.1,3.0,5.9,2.1,Iris-virginica\n"

iris_df = read_csv_or_sample(IRIS_PATH, sample_iris)
print("Raw rows (head):")
print(iris_df.head().to_string(index=False))

df = synthesize_from_sample(iris_df, n=450, noise=0.08, seed=11)

y = df["classification"].astype(str)
X = df.drop(columns=["classification"])

pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, random_state=0)),
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0, stratify=y)

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

acc = accuracy_score(y_test, pred)
print(f"Accuracy (multi-class): {acc:.3f}")
print("\nClassification report:")
print(classification_report(y_test, pred, zero_division=0))


Raw rows (head):
 sepal_length  sepal_width  petal_length  petal_width classification
          5.4          3.7           1.5          0.2    Iris-setosa
          4.8          3.4           1.6          0.2    Iris-setosa
          4.8          3.0           1.4          0.1    Iris-setosa
          4.3          3.0           1.1          0.1    Iris-setosa
          5.8          4.0           1.2          0.2    Iris-setosa
Accuracy (multi-class): 0.948

Classification report:
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        42
Iris-versicolor       0.97      0.86      0.92        44
 Iris-virginica       0.89      0.98      0.93        49

       accuracy                           0.95       135
      macro avg       0.95      0.95      0.95       135
   weighted avg       0.95      0.95      0.95       135



## 7. Ranking / Learning-to-Rank baseline

### 7.1 What makes ranking different?

In classification, each example is independent and has a label.

In ranking, you typically have *queries* (users, sessions, contexts) and a set of candidate items.
You must order items to maximize utility at the top of the list.

### 7.2 Pointwise vs pairwise vs listwise

- **Pointwise:** predict a relevance score per item, then sort
- **Pairwise:** learn preferences between pairs (A should rank above B)
- **Listwise:** optimize a list-level objective directly

We implement a pointwise baseline because it is simple and widely used as a first step.

### 7.3 Evaluation with NDCG

NDCG emphasizes the beginning of the ranked list:

$$
\mathrm{DCG}@k = \sum_{i=1}^{k} \frac{2^{rel_i}-1}{\log_2(i+1)}, \quad
\mathrm{NDCG}@k = \frac{\mathrm{DCG}@k}{\mathrm{IDCG}@k}
$$

We'll create a synthetic relevance label from available features and compute NDCG@10 and NDCG@50.


In [7]:
# Ranking dataset: listings (Airbnb-style)
LIST_PATH = "../../../Datasets/Regression/listings.csv"
sample_listings = "id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license\n13913,Holiday London DB Room Let-on going,54730,Alina,,Islington,51.56861,-0.1127,Private room,57,1,51,2025-02-09,0.29,3,344,10,\n15400,Bright Chelsea  Apartment. Chelsea!,60302,Philippa,,Kensington and Chelsea,51.4878,-0.16813,Entire home/apt,180,4,96,2024-04-28,0.52,1,11,2,\n17402,Very Central Modern 3-Bed/2 Bath By Oxford St W1,67564,Liz,,Westminster,51.52195,-0.14094,Entire home/apt,510,3,56,2024-02-19,0.33,5,293,0,\n24328,Battersea live/work artist house,41759,Joe,,Wandsworth,51.47072,-0.16266,Entire home/apt,213,90,94,2022-07-19,0.54,1,194,0,\n31036,Bright  compact 1 Bedroom Apartment Brick Lane,133271,Hendryks,,Tower Hamlets,51.52425,-0.06997,Entire home/apt,100,2,126,2025-02-20,0.70,8,353,3,\n"

list_df = read_csv_or_sample(LIST_PATH, sample_listings)
print("Raw rows (head):")
print(list_df.head().to_string(index=False))

df = synthesize_from_sample(list_df, n=800, noise=0.06, seed=17)

# Convert and clip numeric columns to keep them in reasonable ranges (prevents invalid log1p)
num_cols_all = ["price", "minimum_nights", "number_of_reviews", "reviews_per_month",
                "calculated_host_listings_count", "availability_365"]
for c in num_cols_all:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["price"] = df["price"].fillna(df["price"].median()).clip(lower=0)
df["reviews_per_month"] = df["reviews_per_month"].fillna(0.0).clip(lower=0)
df["number_of_reviews"] = df["number_of_reviews"].fillna(0.0).clip(lower=0)
df["availability_365"] = df["availability_365"].fillna(df["availability_365"].median()).clip(lower=0)
df["minimum_nights"] = df["minimum_nights"].fillna(df["minimum_nights"].median()).clip(lower=1)
df["calculated_host_listings_count"] = df["calculated_host_listings_count"].fillna(1.0).clip(lower=1)

df["room_type"] = df["room_type"].astype(str)
df["neighbourhood"] = df["neighbourhood"].astype(str)

# Synthetic relevance label (for demonstration only)
score = (0.6*np.log1p(df["number_of_reviews"]) + 0.3*np.log1p(df["availability_365"]) - 0.002*df["price"])
qs = score.quantile([0.25, 0.5, 0.75]).values
df["relevance"] = np.digitize(score, qs).astype(int)

def ndcg_at_k(y_true, y_score, k=10):
    y_true = np.asarray(y_true)
    y_score = np.asarray(y_score)
    order = np.argsort(y_score)[::-1][:k]
    gains = (2**y_true[order] - 1)
    discounts = 1.0 / np.log2(np.arange(2, len(order)+2))
    dcg = np.sum(gains * discounts)

    ideal_order = np.argsort(y_true)[::-1][:k]
    ideal_gains = (2**y_true[ideal_order] - 1)
    idcg = np.sum(ideal_gains * discounts)
    return float(dcg / idcg) if idcg > 0 else 0.0

y = df["relevance"].astype(int)
X = df[[
    "price", "minimum_nights", "number_of_reviews", "reviews_per_month",
    "calculated_host_listings_count", "availability_365",
    "room_type", "neighbourhood"
]].copy()

num_cols = ["price", "minimum_nights", "number_of_reviews", "reviews_per_month",
            "calculated_host_listings_count", "availability_365"]
cat_cols = ["room_type", "neighbourhood"]

pre = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]), num_cols),
    ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                      ("oh", OneHotEncoder(handle_unknown="ignore"))]), cat_cols)
])

ranker = Pipeline([
    ("pre", pre),
    ("rf", RandomForestRegressor(n_estimators=200, random_state=0, n_jobs=-1, min_samples_leaf=3))
])

# Hold out entire neighbourhoods (group-aware evaluation)
unique_neigh = df["neighbourhood"].unique()
rng = np.random.default_rng(0)
test_neigh = set(rng.choice(unique_neigh, size=max(1, len(unique_neigh)//4), replace=False))
test_mask = df["neighbourhood"].isin(test_neigh)

X_train, X_test = X[~test_mask], X[test_mask]
y_train, y_test = y[~test_mask], y[test_mask]

ranker.fit(X_train, y_train)
y_score = ranker.predict(X_test)

print("NDCG@10:", round(ndcg_at_k(y_test, y_score, k=10), 3))
print("NDCG@50:", round(ndcg_at_k(y_test, y_score, k=50), 3))

top_idx = np.argsort(y_score)[::-1][:10]
ranked = df.loc[X_test.index[top_idx], ["name","neighbourhood","room_type","price","number_of_reviews","availability_365","relevance"]]
print("\nTop-10 ranked items (sample):")
print(ranked.reset_index(drop=True).to_string(index=False))


Raw rows (head):
   id                                             name  host_id host_name  neighbourhood_group          neighbourhood  latitude  longitude       room_type  price  minimum_nights  number_of_reviews last_review  reviews_per_month  calculated_host_listings_count  availability_365  number_of_reviews_ltm  license
13913              Holiday London DB Room Let-on going    54730     Alina                  NaN              Islington  51.56861   -0.11270    Private room   57.0               1                 51  2025-02-09               0.29                               3               344                     10      NaN
15400              Bright Chelsea  Apartment. Chelsea!    60302  Philippa                  NaN Kensington and Chelsea  51.48780   -0.16813 Entire home/apt    NaN               4                 96  2024-04-28               0.52                               1                11                      2      NaN
17402 Very Central Modern 3-Bed/2 Bath By Oxford St W

## 8. Forecasting example: daily earthquake magnitude

### 8.1 Forecasting is supervised, but time-dependent

Forecasting predicts $y_{t+h}$ from information up to time $t$.
Even though it is supervised, time dependence changes the rules:

- You must not train on future data.
- You should validate using chronological splits.

### 8.2 Feature engineering via lags

A simple approach is to aggregate by day and create lag features:

- lagged mean magnitude
- lagged event counts

Then train a regression model.

### 8.3 Rolling evaluation

We use `TimeSeriesSplit` to emulate a rolling-window evaluation.
This is not perfect (real systems may have a gap between train and test), but it's much safer than shuffling.

Let's implement it.


In [8]:
# Forecasting dataset: earthquake
EQ_PATH = "../../../Datasets/Regression/earthquake.csv"
sample_earthquake = "date,time,latitude,longitude,depth,magnitude\n2008-11-01,00:31:25,-0.6,98.89553,20.0,2.99\n2008-11-02,01:34:29,-6.61,129.38722,30.1,5.51\n2008-11-03,01:38:14,-3.65,127.99068,5.0,3.54\n2008-11-04,02:20:05,-4.2,128.097,5.0,2.42\n2008-11-05,02:32:18,-4.09,128.20047,10.0,2.41\n"

eq_df = read_csv_or_sample(EQ_PATH, sample_earthquake)
print("Raw rows (head):")
print(eq_df.head().to_string(index=False))

# If you only have a few sample rows, create a multi-day synthetic series for demonstration.
if len(eq_df) < 100:
    df = synthesize_earthquake_timeseries(eq_df, days=220, avg_events_per_day=4, seed=19)
else:
    df = eq_df.copy()

df["timestamp"] = pd.to_datetime(df["date"].astype(str) + " " + df["time"].astype(str), errors="coerce")
df = df.dropna(subset=["timestamp"]).sort_values("timestamp")

daily = df.set_index("timestamp").resample("D")["magnitude"].agg(["mean", "count"]).reset_index()
daily.columns = ["date", "mean_magnitude", "event_count"]

for lag in [1, 2, 3, 7]:
    daily[f"lag_mean_{lag}"] = daily["mean_magnitude"].shift(lag)
    daily[f"lag_count_{lag}"] = daily["event_count"].shift(lag)

daily = daily.dropna().reset_index(drop=True)

y = daily["mean_magnitude"]
X = daily.drop(columns=["date", "mean_magnitude"])

tscv = TimeSeriesSplit(n_splits=5)
model = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc", StandardScaler()),
    ("ridge", Ridge(alpha=1.0, random_state=0)),
])

maes = []
for train_idx, test_idx in tscv.split(X):
    model.fit(X.iloc[train_idx], y.iloc[train_idx])
    pred = model.predict(X.iloc[test_idx])
    maes.append(mean_absolute_error(y.iloc[test_idx], pred))

print("TimeSeries CV MAE (mean):", round(float(np.mean(maes)), 4))
print("TimeSeries CV MAE (std): ", round(float(np.std(maes)), 4))

model.fit(X, y)
daily["pred_next_mean_mag"] = model.predict(X)

print("\nLast 5 days (actual vs predicted mean magnitude):")
print(daily[["date", "mean_magnitude", "pred_next_mean_mag", "event_count"]].tail(5).to_string(index=False))


Raw rows (head):
      date     time  latitude  longitude  depth  magnitude
2008-11-01 00:31:25     -0.60   98.89553   20.0       2.99
2008-11-01 01:34:29     -6.61  129.38722   30.1       5.51
2008-11-01 01:38:14     -3.65  127.99068    5.0       3.54
2008-11-01 02:20:05     -4.20  128.09700    5.0       2.42
2008-11-01 02:32:18     -4.09  128.20047   10.0       2.41
TimeSeries CV MAE (mean): 0.2143
TimeSeries CV MAE (std):  0.0214

Last 5 days (actual vs predicted mean magnitude):
      date  mean_magnitude  pred_next_mean_mag  event_count
2022-09-22        3.388400            3.467577           25
2022-09-23        3.507105            3.343235           38
2022-09-24        3.461333            3.539307           15
2022-09-25        3.242609            3.466240           23
2022-09-26        3.645238            3.492252           21


## 9. Summary and key takeaways

1. **Start from the target and decision.**  
   Algorithms come later.

2. **Metrics are task-dependent.**  
   - Regression: MAE/RMSE
   - Classification: F1/AUC (and calibration if needed)
   - Ranking: NDCG@k and other top-$k$ measures
   - Forecasting: horizon-aware errors with chronological splits

3. **Validation protocol is part of the task definition.**  
   Time series requires time-aware evaluation.

4. **Baselines matter.**  
   A correct simple baseline can beat a complex model trained and evaluated incorrectly.

---

## 10. Exercises

1. In the regression section, replace the linear model with `RandomForestRegressor` and compare MAE/RMSE.
2. In the classification section, tune the threshold $\tau$ to maximize F1 on a validation split.
3. Implement Precision@k for the ranking section and compare it with NDCG@k.
4. Try expanding-window splits for forecasting and compare errors.

---

If you'd like, we can add a short "mini-project" at the end of this notebook:
given a business scenario, you decide the task type, define $y$, choose the metric,
and write the minimum viable modeling pipeline.
